## Implementação do modelo Selecionado

In [12]:
import collections
import numpy as np
import pandas as pd
import pickle
import difflib
import random
from plotly.offline import iplot
from surprise import SVD, KNNBasic, CoClustering
from surprise import Dataset
from surprise import Reader
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import GridSearchCV
from surprise.model_selection import cross_validate
from surprise.model_selection import train_test_split
from surprise import Dataset
from surprise import Reader

Carregando dados e o modelo escolhido 

In [2]:
ratings_df = pd.read_csv("ratings.csv")
ratings_df =  ratings_df.drop(['timestamp'], axis=1)
movies = pd.read_csv("movies.csv",encoding="Latin1")

reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_df[['userId', 'movieId', 'rating']], reader)

In [16]:
model = pickle.load( open( "model_selected.p", "rb" ) )
model

Avaliação Modelo

In [18]:
trainset, testset = train_test_split(data, test_size=.25)
model.fit(trainset)
predictions = model.test(testset)

print(accuracy.rmse(predictions))
print(accuracy.mae(predictions))

RMSE: 0.8598
0.8597919487647819
MAE:  0.6616
0.6615936463895931


Criando algorítmo para recomendação

In [43]:

def get_movie_id(movie_title):
    
    """
    Gets the book ID for a book title based on the closest match in the metadata dataframe.
    """
    
    existing_titles = list(movies['title'].values)
    closest_titles = difflib.get_close_matches(movie_title, existing_titles)
    movie_id = movies[movies['title'] == closest_titles[0]]['movieId'].values[0]
    return movie_id

def get_movie_info(movie_id):
    movie_info = movies[movies['movieId'] == movie_id]
    return movie_info.to_dict(orient='records')

def predict_review(user_id, movie_title, model):
    
    movie_id = get_movie_id(movie_title)
    review_prediction = model.predict(uid=user_id, iid=movie_id)
    return review_prediction.est

def generate_recommendation(user_id, model,movies, thresh=3.7):

    # top_25 = ratings_df['movieId'].value_counts()[:25].index.tolist()

    # movie_list = movies[movies['movieId'].isin(top_25)].reset_index(drop=True)
    # review_movies(movie_list)

    movies = list(movies['title'].values)
    random.shuffle(movies)
    
    for movie_title in movies:
        rating = predict_review(user_id, movie_title, model)
        if rating >= thresh:
            movie_id = get_movie_id(movie_title)
            return get_movie_info(movie_id)


In [44]:
def review_movies(movie_list, id):
    movies = movie_list
    reviews = pd.DataFrame([], columns=['userId', 'movieId', 'rating'])
    for i in range(10):
        movie_index = random.randint(0, len(movies))
        rating = input('Nota Filme ' + movies.loc[[movie_index]]['title'].values[0] + ' (0 a 5)')
        print('Nota Filme ', movies.loc[[movie_index]]['title'].values[0], ' (0 a 5)',  rating)
        
        if(rating == ''):
            rating = None
        else:
            rating = float(rating)
            
        user_array = [id, movies.loc[[movie_index]]['movieId'].values[0], rating ]
        user_df = pd.DataFrame([user_array], columns=['userId', 'movieId', 'rating'])
        reviews = reviews.append(user_df, ignore_index=True)
        movies = movies.drop(movie_index).reset_index(drop=True)
    return reviews

In [45]:
def new_user(data, model):
    top_25 = data['movieId'].value_counts()[:100].index.tolist()
    movie_list = movies[movies['movieId'].isin(top_25)].reset_index(drop=True)
    id = data['userId'].values[-1]+1
    ratings = review_movies(movie_list,id )
    data=data.append(ratings, ignore_index = True)
    reader = Reader(rating_scale=(1, 5))
    data_ds = Dataset.load_from_df(data[['userId', 'movieId', 'rating']], reader)
    model.fit(data_ds.build_full_trainset())
    return id


Exemplo de uso

In [48]:
new_user_id = new_user(ratings_df, model)
generate_recommendation(new_user_id, model, movies, 3.5)


Nota Filme  Die Hard (1988)  (0 a 5) 3
Nota Filme  Terminator 2: Judgment Day (1991)  (0 a 5) 4
Nota Filme  Lord of the Rings: The Return of the King, The (2003)  (0 a 5) 5
Nota Filme  Pirates of the Caribbean: The Curse of the Black Pearl (2003)  (0 a 5) 3
Nota Filme  Breakfast Club, The (1985)  (0 a 5) 4
Nota Filme  Spider-Man (2002)  (0 a 5) 5
Nota Filme  Batman Begins (2005)  (0 a 5) 1
Nota Filme  Ace Ventura: Pet Detective (1994)  (0 a 5) 4
Nota Filme  Princess Bride, The (1987)  (0 a 5) 5
Nota Filme  Apollo 13 (1995)  (0 a 5) 4


[{'movieId': 47640, 'title': 'Beerfest (2006)', 'genres': 'Comedy'}]